# Flexion Angle Detection

The following cells are able to plot a human pose estimate on a pre-recorded video stream and get the angle between specific joints of the right arm.

## Key Aspects
- Runs on OpenCV - machine learning applicable library for computer vision
- Real time application
- Can run on CPU (later to be optimized for Nvidia GPU cores)



In [1]:
#importing required packages and dependancies
import cv2
import numpy as np
import matplotlib as plt
import math
import time
import mediapipe as mp
import pandas as pd

## Defining a class poseDetector with predefined joints pose, position and angle functions
The cell below is to initiate the pose detector class as directed from MediaPipe with various parameters in the class for pose detection.

We can have different classes for face, hand, pose and the angle but all of them have associated attributes hence a single class called poseDetector with diferent functions to be called later on in the notebook.

In [2]:
class poseDetector():

    def __init__(self, mode=False, complexity=1, smooth_landmarks=True, 
                 segmentation=False, smooth_segmentation=True, detectionCon=0.5, trackCon=0.5):
        
        self.mode = mode
        self.complexity = complexity
        self.smooth_landmarks = smooth_landmarks
        self.segmentation = segmentation
        self.smooth_segmentation = smooth_segmentation
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, 
                                     self.complexity, 
                                     self.smooth_landmarks,
                                     self.segmentation,
                                     self.smooth_segmentation,
                                     self.detectionCon, 
                                     self.trackCon)
    #function to find the pose    
    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks,
                                           self.mpPose.POSE_CONNECTIONS)
        return img
    
    #funtion to return the position of all the joints in the image/video
    def findPosition(self, img, draw=True):
        self.lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                # print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return self.lmList

    #function to return the angle between joints p1,p2,p3
    def findAngle(self, img, p1, p2, p3, draw=True):
        # Get the landmarks/positions of the joints
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        x3, y3 = self.lmList[p3][1:]

        # Calculate the Angle between the mapped joints
        angle = math.degrees(math.atan2(y3 - y2, x3 - x2) -
                             math.atan2(y1 - y2, x1 - x2))
        if angle < 0:
            angle += 360
        # print(angle)

        # Draw the selected joints and the line joining them
        if draw:
            #highlighting the skeleton connecting the joints
            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            #highlighting the joints with bigger circles
            cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)
            # Drawing the value of the angle on the highlighted joint
            cv2.putText(img, str(int(angle)), (x2 - 50, y2 + 50),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        return angle

## Pose on image from poseDetector
The cell below is capable of plotting the human pose on images/videos of human beings in different postures.

The poseDetector class defined in the cells above is imported and a defined function is used to plot the human pose on the image/video provided.

In [ ]:
cap = cv2.VideoCapture("TestVideo/FlexTest.mp4")
detector = poseDetector() #from class poseDetector in cells above 

while True:
    #for video input
    #success, img = cap.read()
    #for image input
    img = cv2.imread("TestVideo/test4.jpg")
    
    img = cv2.resize(img, (900,700))
    #img = cv2.flip(img, 0)

    img = detector.findPose(img, False)
    lmList = detector.findPosition(img, False)
    #print(jointsList)
    
    #Defining the 3 joints whose angle you need to find
    if len(jointsList) != 0:
        #left leg
        detector.findAngle(img, 23, 25, 27)
        #right leg
        #detector.findAngle(img, 24, 26, 28)
        #right arm
        #detector.findAngle(img, 12, 14, 16)
        #left arm
        detector.findAngle(img, 11, 13, 15)
        
    cv2.imshow("image", img)
    cv2.waitKey(1)
    

